In [1]:
from TestCase import TestCase
import numpy as np
from MonteCarloTreeSearch_Source.build.PyMCTS import MonteCarloTreeSearch

Below we define all the parameters of our problem and generate a hypothetical test case. Here we are keeping it simple, in the problem statement demand_max is 400, and for the algorithm to give intelligent meaningful results we should set nmbr_brnch_myTurn, nmbr_brnch_wrldTurn in the hundereds and nmbr_simulations_per_rollout in the thousands, however that would take hours(at-least). Here I keep it small as I just wanna see if the prototype works.

In [6]:
demand_min = 10
demand_max = 20
nmbr_strs = 3
nmbr_prdcts = 4
time_frm = 10
DC_cpcty = 200
truck_capacity = 20
nmbr_brnch_wrldTurn = 10
nmbr_brnch_myTurn = 10
nmbr_simulations_per_rollout = 10
factory_production_limit = 200
exploration_factor = 2
trk_thrpt_rti_cnst = 1
wareHouseStatePy = np.zeros(nmbr_prdcts, dtype=np.int32)
tc = TestCase( nmbr_strs, nmbr_prdcts, DC_capacity = DC_cpcty,demand_range_min = demand_min,
              demand_range_max = demand_max ,timeframe = time_frm)
demand_prb_dstrbutnPy = tc.getData(130, simplex_reduction_factor = 0.01)
    

clipping 0(0,0) out of 120 values
66.66666666666667
33.3333333333333360.0



In the two player game we are playing the world makes a move first and then we make a move, i.e. we have a demand probability for 90 days. However since (mathematically) factory to DC is instantaneus and factory product is also instant, it makes no sense mathematically to not make the decision to simulate once we get the very first demand. So we make a first demand

In [3]:
demand_right_now = tc.generateStochasticInstantDemand(demand_prb_dstrbutnPy, demand_min)

In [4]:
demand_right_now

array([[13, 10, 17, 11],
       [13, 17, 13, 19],
       [14, 18, 15, 14]], dtype=int32)

Now we initialise the tree. It will run the simulation and will have a result ready for us.

In [7]:
tree = MonteCarloTreeSearch(demand_min,demand_max,nmbr_strs, nmbr_prdcts,time_frm,DC_cpcty,truck_capacity,
   nmbr_brnch_wrldTurn, nmbr_brnch_myTurn,nmbr_simulations_per_rollout, factory_production_limit, 
                            exploration_factor,trk_thrpt_rti_cnst, wareHouseStatePy, demand_prb_dstrbutnPy, demand_right_now)

Simulation  0  of  10
Progress ::  0 %
demand_min_ :: 10
demand_max_ :: 20
nmbr_strs_ :: 3
nmbr_prdcts_ :: 4
time_frm_ :: 10
DC_cpcty_ :: 200
truck_capacity_ :: 20
factory_production_limit_ :: 200
nmbr_brnch_wrldTurn_ :: 10
nmbr_brnch_myTurn_ :: 10
nmbr_simulations_per_rollout_ :: 10
exploration_factor_ :: 2
trk_thrpt_rti_cnst_ :: 1
Progress ::  10 %
Progress ::  20 %
Progress ::  30 %
Progress ::  40 %
Progress ::  50 %
Progress ::  60 %
Progress ::  70 %
Progress ::  80 %
Progress ::  90 %
Simulation  1  of  10
Progress ::  0 %
Progress ::  10 %
Progress ::  20 %
Progress ::  30 %
Progress ::  40 %
Progress ::  50 %
Progress ::  60 %
Progress ::  70 %
Progress ::  80 %
Progress ::  90 %
Simulation  2  of  10
Progress ::  0 %
Progress ::  10 %
Progress ::  20 %
Progress ::  30 %
Progress ::  40 %
Progress ::  50 %
Progress ::  60 %
Progress ::  70 %
Progress ::  80 %
Progress ::  90 %
Simulation  3  of  10
Progress ::  0 %
Progress ::  10 %
Progress ::  20 %
Progress ::  30 %
Progress

We can keep calling the MCTS method again and again to keep explore the tree, while this functionality is present, this is used in advasarial 2 player games like chees. Using it is pointless here, though if you want to you can keep exploring the tree more and more.

In [ ]:
tree()

In [8]:
(factory_produce, trucks) = tree.CurrentBestSolution
factory_produce, trucks

([44, 46, 43, 49],
 [(0, [9, 9, 2, 0]),
  (0, [0, 0, 12, 8]),
  (1, [11, 9, 0, 0]),
  (1, [0, 8, 12, 0]),
  (1, [0, 0, 1, 19]),
  (2, [14, 6, 0, 0]),
  (2, [0, 11, 9, 0]),
  (2, [0, 0, 6, 14])])

Here is the result, with factory produce being a list where the ith index means how much of prouduct i to produce, and trucks being a list of trucks. Where each truck is a tuple of (store, truck plan). The store is which store the truck is going to and the truck plan it a list where the ith element is how much of the ith product is being taken to that store. 